# 0.0 - Imports

In [18]:
# Bibliotecas
import pandas  as pd
import numpy   as np
import seaborn as sns

from statsmodels.sandbox.stats.multicomp import multipletests
from itertools                           import combinations
from statsmodels.stats                   import gof, power, proportion
from scipy                               import stats

## 0.1 Carregamento dos Dados

In [4]:
# Armazenamento dos Dados
dataframe = {'page_variant_button': ['interact', 'connect', 'learn', 'help', 'services'], 
             'visitantes': [10283, 2742, 2747, 3180, 2064],
             'clicks':[3714, 1587, 1652, 1717, 1348],
             'clicks_button': [42, 53, 21, 38, 45]}

# Armazenamento dos Dados em um DataFrame
df_raw = pd.DataFrame(dataframe)

# Exibição dos dados
df_raw.head()

,page_variant_button,visitantes,clicks,clicks_button
0,interact,10283,3714,42
1,connect,2742,1587,53
2,learn,2747,1652,21
3,help,3180,1717,38
4,services,2064,1348,45


# 1.0 - Escolha do Método

Para o problema que nos foi designado, nós vamos utilizar o teste A/B/C/D para responder a questão de negócio da Universidade de Montana. Então vamos iniciar a estruturação do nosso problema.

**Qual o objetivo do teste?**

- O objetivo do nosso experimento é verificar qual dos botões que a o time TI da universidade proporcionou é mais atrativo, isto é, possui uma maior taxa de clique. No caso, são 4 botões: Connect, Lean, Help e Service.

**Qual o tipo dos dados?**

- Primeiramente, queremos verificar se há alguma diferença entre as taxas de conversões de cliques dos botões fornecidos pelo time de TI e os dados que nós temos disponíveis são discretos.

**Qual a quantidade de dados?**

- São mais de 5 dados.

**Qual o foco do teste?**

- O foco do teste é verificar se existe uma relação entre o nome do botão e a quantidade de cliques sobre esse botão.

**Quantos grupos fazem parte do teste?**

- São 4 grupos que faze parte do teste.

**Qual Inferência utilizar?**

- Vamos usar o teste de independência chi-quadrado.

# 2.0 - Design de Experimento

## 2.1 - Formulação das Hipóteses 

**Determinação da Métrica:** Quantidade de Clicks na página.

**Hipótese Nula:** Não há uma relação entre os títulos dos botões e os cliques.

**Hipótese Alternativa:** Há uma relação entre os títulos dos botões e os cliques.

## 2.2 - Parâmetros para o Tamanho da Amostra 

In [5]:
# Nível de Confiança
nivel_confianca = 0.95

# Nível de Significância
nivel_significancia = 1 - nivel_confianca

# Poder Estatístico
power_stats = 0.80

# 3.0 - Preparação e Coleta dos Dados de Amostragem


In [8]:
# Criação da tabela de contigência
df_raw['no_clicks_button'] = df_raw['clicks'] - df_raw['clicks_button']

# Extraindo somente os dados necessários
table_contingency = df_raw[['page_variant_button', 'clicks_button', 'no_clicks_button']].copy()

# Reajuste de Index
table_contingency = table_contingency.set_index('page_variant_button')

# Exibição da Tabela de Contigência
table_contingency

,clicks_button,no_clicks_button
page_variant_button,,
interact,42,3672
connect,53,1534
learn,21,1631
help,38,1679
services,45,1303


# 4.0 - Teste de Hipótese

In [13]:
# Realizando a Inferência Estatística
chi2, p_value, dof, ex = stats.chi2_contingency(np.array(table_contingency))

# Resultados
print('Chi-Quadrado: {}'.format(chi2))
print('P-Valor: {}'.format(p_value))

if p_value < 0.05:
    print('Concluímos que há uma relação entre os botões e os cliques.')
else:
    print('Concluímos que não há uma relação entre os botões e os cliques.')

Chi-Quadrado: 46.33660181942126
P-Valor: 2.0959498129984567e-09
Concluímos que há uma relação entre os botões e os cliques.


## 4.1 - Post-Hoc

In [11]:
# Vamos realizar a combinação dois a dois de todas as possibilidades.
all_combinations = list(combinations(table_contingency.index, 2))

# lista vazia para armazenar os p_values
p_value_list = list()

for combination in all_combinations:
    
    # Obtendo a tabela de contigencia com os dados de dois a dois.
    new_df = table_contingency[(table_contingency.index == combination[0]) | (table_contingency.index == combination[1])]
    
    # Calculando o p_valor
    chi2, p_value, dof, ex = stats.chi2_contingency(new_df)
    
    # Armazenando os p_valores
    p_value_list.append(p_value)
    
# correção dos p_valores por Bonferroni
reject_list, p_value_correct = multipletests(p_value_list, method='Bonferroni')[:2]

# Exibição dos Resultados

for combination, p_value, p_value_correct, reject in zip(all_combinations, p_value_list, p_value_correct, reject_list):
    
    print('=========================================')
    print('Combinação: {}'.format(combination))
    print('P-valor: {}'.format(p_value))
    print('P-valor corrigido: {}'.format(p_value_correct))
    print('Rejeição da Hipótese Nula: {}'.format(reject))


Combinação: ('interact', 'connect')
P-valor: 5.3676772349808135e-08
P-valor corrigido: 5.367677234980813e-07
Rejeição da Hipótese Nula: True
Combinação: ('interact', 'learn')
P-valor: 0.7616980743361713
P-valor corrigido: 1.0
Rejeição da Hipótese Nula: False
Combinação: ('interact', 'help')
P-valor: 0.0031030587017400212
P-valor corrigido: 0.03103058701740021
Rejeição da Hipótese Nula: True
Combinação: ('interact', 'services')
P-valor: 1.798089447385411e-07
P-valor corrigido: 1.7980894473854111e-06
Rejeição da Hipótese Nula: True
Combinação: ('connect', 'learn')
P-valor: 0.00013292868361715983
P-valor corrigido: 0.0013292868361715984
Rejeição da Hipótese Nula: True
Combinação: ('connect', 'help')
P-valor: 0.06144184057612575
P-valor corrigido: 0.6144184057612575
Rejeição da Hipótese Nula: False
Combinação: ('connect', 'services')
P-valor: 1.0
P-valor corrigido: 1.0
Rejeição da Hipótese Nula: False
Combinação: ('learn', 'help')
P-valor: 0.0508958228881819
P-valor corrigido: 0.5089582288

### Relatório - Hipótese Nula Rejeitada

# 5.0 - Conclusões

Ao realizar o teste chi-quadrado de independência, concluímos que ao comparar o botão "Interact" com o botão "Connect", "Help" e "Services", existe um relação entre os cliques no botão e os títulos, isto é, podemos dizer que o título de um desses botões tem um impacto na taxa de cliques. 

Com essas informações, precisamos saber se algum desses botões são melhores que o Interact. Para isso, será necessário realizar um teste que nos permita realizar a comparação da proporção de taxas entre as duas páginas. Logo, será realizado um novo design de experimento em outro documento para ficar organizado. 

Será realizado o teste dois a dois dos seguintes botões:

1. Botão Interact com o botão Connect.
2. Botão Interact com o botão Help.
3. Botão Interact com o botão Services.

Feito esses testes, poderemos concluir com clareza qual é o melhor botão.